# 1.3 : Shifts of Nurses
### finding the optimal number of nurses from " fau_clinic_shifts.csv " dataset 

## Data Loading, Cleaning and Processing

In [1]:
# import the neccessary libraries
import pandas as pd
import pulp


In [2]:
# import the dataset into a dataframe
df = pd.read_csv("fau_clinic_shifts.csv", index_col=0)

In [3]:
# Preprocess the data
df = df.fillna(0).applymap(lambda x: 1 if x == "X" else x)

In [4]:
df

,Shift 1,Shift 2,Shift 3,Avg_Patient_Number
Time Windows,,,,
6:00-07:00,1,0,0,49.0
07:00 – 08:00,1,0,0,48.0
08:00 – 09:00,1,0,0,42.0
09:00 – 10:00,1,0,0,36.0
10:00 – 11:00,1,0,0,55.0
11:00 – 12:00,1,0,0,51.0
12:00-13:00,1,0,0,47.0
13:00 – 14:00,1,0,0,43.0
14:00 – 15:00,0,1,0,37.0


In [5]:
# create a matrix to show which shift each time window is associated with

shifts = df.drop(index=["Wage rate per 8h shift (EUR)"], columns=["Avg_Patient_Number"]).values

In [6]:
# Extract relevant information from the dataframe
shifts = df.drop(index=["Wage rate per 8h shift (EUR)"], columns=["Avg_Patient_Number"]).values
shift_num = shifts.shape[1]
time_windows = shifts.shape[0]
avg_patient_number = df["Avg_Patient_Number"].values
wages_per_shift = df.loc["Wage rate per 8h shift (EUR)", :].values.astype(int)
service_level = 0.25

In [7]:
# Define the decision variables
num_nurses_inds = range(shift_num)
num_nurses = pulp.LpVariable.dicts("num_nurses", num_nurses_inds, lowBound=0, cat="Integer")

## Creating LP Problem to determine the result out of it.

In [8]:
# Create the LP problem
lp_problem = pulp.LpProblem("scheduling_nurses", pulp.LpMinimize)

# Define the objective function
lp_problem += pulp.lpSum([wages_per_shift[shift_ind] * num_nurses[shift_ind] for shift_ind in num_nurses_inds])

# Define the constraints
for t in range(time_windows):
    lp_problem += pulp.lpSum([shifts[t, i] * num_nurses[i] for i in num_nurses_inds]) >= (avg_patient_number[t] * service_level)

# Solve the LP problem
lp_problem.solve()


1

In [9]:
# Print the results
print("Status:", pulp.LpStatus[lp_problem.status], "\n")

for shift_ind in num_nurses_inds:
    shift_number = shift_ind + 1
    num_nurses_needed = int(num_nurses[shift_ind].value())
    print(f"The number of nurses needed for shift {shift_number} is {num_nurses_needed} nurses")

Status: Optimal 

The number of nurses needed for shift 1 is 14 nurses
The number of nurses needed for shift 2 is 10 nurses
The number of nurses needed for shift 3 is 13 nurses
